ARTIFICIAL NEURAL NETWORK (ANN)

Case Study: SONAR – Mine vs Rock Classification

STEP 1: Import Required Libraries

In [2]:
!pip install scikeras


In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


Step 2: Load Dataset

In [7]:
data = pd.read_csv("/content/sonardataset.csv", header=None)

X = data.iloc[:, 0:60]   # Features
y = data.iloc[:, 60]    # Labels (M or R)

print("Dataset shape:", data.shape)
print("Class distribution:\n", y.value_counts())


Dataset shape: (209, 61)
Class distribution:
 60
M    111
R     97
Y      1
Name: count, dtype: int64


Step 3: Encode Labels

In [8]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)   # M=1, R=0


Step 4: Normalize Data

In [10]:
scaler = StandardScaler()
X = X.iloc[1:].values # Remove the header row and convert to numpy array
X = scaler.fit_transform(X)

Step 5: Train-Test Split

In [12]:
y = y[1:] # Remove the first element from y to match X's number of samples

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

Step 6: Build ANN Model

In [13]:
model = Sequential()
model.add(Dense(32, input_dim=60, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

Step 7: Train Model

In [14]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_test, y_test))


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.6655 - loss: 0.6249 - val_accuracy: 0.6154 - val_loss: 0.6284
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6559 - loss: 0.6130 - val_accuracy: 0.7500 - val_loss: 0.5812
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6714 - loss: 0.5535 - val_accuracy: 0.7885 - val_loss: 0.5438
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7401 - loss: 0.5158 - val_accuracy: 0.8269 - val_loss: 0.5113
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7788 - loss: 0.4821 - val_accuracy: 0.8654 - val_loss: 0.4803
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8575 - loss: 0.4020 - val_accuracy: 0.8462 - val_loss: 0.4506
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8866 - loss: 0.3907 - val_accuracy: 0.8462 - val_loss: 0.4261
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9008 - loss: 0.3547 - val_accuracy: 0.

Step 8: Evaluate Model

In [15]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Accuracy: 0.9038461538461539

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91        28
           1       0.91      0.88      0.89        24

    accuracy                           0.90        52
   macro avg       0.90      0.90      0.90        52
weighted avg       0.90      0.90      0.90        52



Step 9: Hyperparameter Tuning

In [16]:
# Try different neuron counts
for neurons in [16, 32, 64]:
    print("\nTraining model with", neurons, "neurons")

    model = Sequential()
    model.add(Dense(neurons, input_dim=60, activation='relu'))
    model.add(Dense(neurons//2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    y_pred = (model.predict(X_test) > 0.5).astype(int)

    print("Accuracy:", accuracy_score(y_test, y_pred))



Training model with 16 neurons


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Accuracy: 0.8653846153846154

Training model with 32 neurons


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy: 0.8846153846153846

Training model with 64 neurons
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Accuracy: 0.8653846153846154


        Conclusion

ANN successfully learns sonar signal patterns

Deep layers extract frequency relationships

High accuracy achieved after tuning

Model is robust, scalable, and reliable

Suitable for real-time underwater object classification